In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Initialiser les DataFrames
df_programmes = pd.DataFrame()
df_reunions = pd.DataFrame()
df_courses = pd.DataFrame()
df_participations = pd.DataFrame()

id_course = 0
id_reunion = 0
id_participation = 0

# Date du premier enregistrement 19-02-2013
# id_programme = 19022013

# Date du premier enregistrement 19-02-2013
start_date = datetime.strptime("19022013", "%d%m%Y")
end_date = datetime.strptime("01032013", "%d%m%Y")
# end_date = datetime.now()

current_date = start_date

while current_date <= end_date:
    id_programme = current_date.strftime("%d%m%Y")

    # URL avec la date de début
    url = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}"

    # Récupération du Json
    response = requests.get(url)

    if response.status_code == 200:
        json_data = response.json()

        #------------------------------------------------------------------- Construction de la table programme -------------------------------------------------------------------
        # Extraire les informations du programme
        date_programme_timestamp = json_data["programme"]["date"] // 1000
        date_programme = datetime.fromtimestamp(date_programme_timestamp).date()

        # Format personnalisé de la date en français
        date_programme_fr = date_programme.strftime("%Y-%m-%d")

        # Afficher les informations du programme
        # print(f"Id du programme : {id_programme} | Date du programme : {date_programme_fr}")
        df_programmes = pd.concat([df_programmes, pd.DataFrame([{"id_programme": id_programme, "date_programme": date_programme_fr}])], ignore_index=True)
        #------------------------------------------------------------------- Construction de la table programme -------------------------------------------------------------------

        #------------------------------------------------------------------- Construction de la table reunions --------------------------------------------------------------------

        for reunion in json_data["programme"]["reunions"]:
            # id_reunion est date + numero de la reunion exemple 19022013R1
            id_reunion += 1

            discipline_mere = ', '.join(reunion.get("disciplinesMere", []))
            specialites = reunion.get("specialites", [])

            # Assurez-vous d'avoir suffisamment d'éléments, sinon remplissez avec None
            specialites_padded = specialites + [None] * (4 - len(specialites))

            reunion_data = {
                "id_reunion": id_reunion,
                "id_programme": id_programme,
                "num_officiel": reunion.get("numOfficiel"),
                "nature": reunion.get("nature"),
                "code_hippodrome": reunion.get("hippodrome", {}).get("code"),
                "libelle_court_hippodrome": reunion.get("hippodrome", {}).get("libelleCourt"),
                "libelle_long_hippodrome": reunion.get("hippodrome", {}).get("libelleLong"),
                "code_pays": reunion["pays"].get("code"),
                "libelle_pays": reunion["pays"].get("libelle"),
                "audience": reunion.get("audience"),
                "statut": reunion.get("statut"),
                "disciplines_mere": discipline_mere,
                "specialite_1": specialites_padded[0],
                "specialite_2": specialites_padded[1],
                "specialite_3": specialites_padded[2],
                "specialite_4": specialites_padded[3],
                "meteo_nebulosite_code": reunion.get("meteo", {}).get("nebulositeCode"),
                "meteo_nebulosite_Libelle_Court": reunion.get("meteo", {}).get("nebulositeLibelleCourt"),
                "meteo_nebulosite_Libelle_Long": reunion.get("meteo", {}).get("nebulositeLibelleLong"),
                "meteo_temperature": reunion.get("meteo", {}).get("temperature"),
                "meteo_force_vent": reunion.get("meteo", {}).get("forceVent"),
                "meteo_direction_vent": reunion.get("meteo", {}).get("directionVent")
            }

            df_reunions = pd.concat([df_reunions, pd.DataFrame([reunion_data])], ignore_index=True)
            #------------------------------------------------------------------- Construction de la table reunions --------------------------------------------------------------------




            #------------------------------------------------------------------- Construction de la table courses --------------------------------------------------------------------
            num_reunion = reunion_data['num_officiel']
            url_courses = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}/R{num_reunion}"

            # Récupération du Json
            response_courses = requests.get(url_courses)

            if response_courses.status_code == 200:
                json_data_courses = response_courses.json()
            

                for course in json_data_courses["courses"]:
                    id_course += 1
                    heureDepart_timestamp = course.get("heureDepart", 0) // 1000

                    # Gérer les timestamps négatifs
                    if heureDepart_timestamp < 0:
                        heureDepart = None
                    else:
                        heureDepart = datetime.fromtimestamp(heureDepart_timestamp).time()

                    # Convertir dureeCourse de millisecondes à un format lisible
                    duree_course_ms = course.get("dureeCourse", 0)
                    duree_course_minutes, duree_course_seconds = divmod(duree_course_ms // 1000, 60)
                    duree_course_formatted = f"{duree_course_minutes}m {duree_course_seconds}s"

                    # Extraction des incidents
                    incidents_list = course.get("incidents", [])
                    incidents_type = [incident.get("type") for incident in incidents_list] if isinstance(incidents_list, list) else []
                    incidents_participants = [incident.get("numeroParticipants") for incident in incidents_list] if isinstance(incidents_list, list) else []

                    # Conversion des listes en chaînes de caractères
                    incidents_type_str = ' | '.join(incidents_type)
                    incidents_participants_str = ' | '.join([', '.join(map(str, participants)) for participants in incidents_participants])

                    specialites = reunion.get("specialites", [None, None])  # Deux spécialités ou None
                    specialite_1 = specialites[0] if len(specialites) > 0 else None
                    specialite_2 = specialites[1] if len(specialites) > 1 else None

                    # Récupération des ordres d'arrivée
                    ordre_arrivee = course.get("ordreArrivee", [])
                    ordre_arrivee_padded = [ordre[0] if isinstance(ordre, list) else ordre for ordre in (ordre_arrivee + [None] * (5 - len(ordre_arrivee)))]

                    course_data = {
                        "id_course": id_course,
                        "id_reunion": id_reunion,
                        "libelle": course.get("libelle"),
                        "libelle_court": course.get("libelleCourt"),
                        "heure_depart": heureDepart,
                        "parcours": course.get("parcours"),
                        "distance": course.get("distance"),
                        "distance_unit": course.get("distanceUnit"),
                        "corde": course.get("corde"),
                        "discipline": course.get("discipline"),
                        "specialite_1": specialite_1,
                        "specialite_2": specialite_2,
                        "condition_sexe": course.get("conditionSexe"),
                        "conditions": course.get("conditions"),
                        "statut": course.get("statut"),
                        "categorie_statut": course.get("categorieStatut"),
                        "duree_course": course.get("dureeCourse"),
                        "duree_course_en_minute":duree_course_formatted,
                        "montant_prix": course.get("montantPrix"),
                        "grand_prix_national_trot": course.get("grandPrixNationalTrot"),
                        "nombre_declares_partants": course.get("nombreDeclaresPartants"),
                        "montant_total_offert": course.get("montantTotalOffert"),
                        "premier": ordre_arrivee_padded[0],
                        "montant_offert_1er": course.get("montantOffert1er"),
                        "deuxieme": ordre_arrivee_padded[1],
                        "montant_offert_2eme": course.get("montantOffert2eme"),
                        "troisieme": ordre_arrivee_padded[2],
                        "montant_offert_3eme": course.get("montantOffert3eme"),
                        "quatrieme": ordre_arrivee_padded[3],
                        "montant_offert_4eme": course.get("montantOffert4eme"),
                        "cinquieme": ordre_arrivee_padded[4],
                        "montant_offert_5eme": course.get("montantOffert5eme"),
                        "incidents_type": incidents_type_str,
                        "incidents_participants": incidents_participants_str,
                    }
                    df_courses = pd.concat([df_courses, pd.DataFrame([course_data])], ignore_index=True)
                    #------------------------------------------------------------------- Construction de la table courses --------------------------------------------------------------------


                    #------------------------------------------------------------------- Construction de la table participations -------------------------------------------------------------
                    num_course = course.get("numOrdre")
                    url_participants = f"https://offline.turfinfo.api.pmu.fr/rest/client/7/programme/{id_programme}/R{num_reunion}/C{num_course}/participants"

                    # Récupération du Json
                    response_participants = requests.get(url_participants)

                    if response_participants.status_code == 200:
                        json_data_participants = response_participants.json()

                        for participant in json_data_participants["participants"]:
                            id_participation += 1

                            # Convertir dureeCourse de millisecondes à un format lisible
                            temps_obtenu_ms = participant.get("tempsObtenu", 0)
                            temps_obtenu_minutes, temps_obtenu_seconds = divmod(temps_obtenu_ms // 1000, 60)
                            temps_obtenu_formatted = f"{temps_obtenu_minutes}m {temps_obtenu_seconds}s"

                            participation_data = {
                                "id_participation": id_participation,
                                "id_course": id_course,
                                "nom": participant.get("nom"),
                                "numero_cheval": participant.get("numPmu"),
                                "age": participant.get("age"),
                                "sexe": participant.get("sexe"),
                                "race": participant.get("race"),
                                "statut": participant.get("statut"),
                                "proprietaire": participant.get("proprietaire"),
                                "entraineur": participant.get("entraineur"),
                                "driver": participant.get("driver"),
                                "driverChange": participant.get("driverChange"),
                                "code_robe": participant.get("robe", {}).get("code"),
                                "libelleCourt_robe": participant.get("robe", {}).get("libelleCourt"),
                                "libelleLong_robe": participant.get("robe", {}).get("libelleLong"),
                                "nombre_courses": participant.get("nombreCourses"),
                                "nombre_victoires": participant.get("nombreVictoires"),
                                "nombre_places": participant.get("nombrePlaces"),
                                "nom_pere": participant.get("nomPere"),
                                "nom_mere": participant.get("nomMere"),
                                "ordre_arrivee": participant.get("ordreArrivee"),
                                "jument_pleine": participant.get("jumentPleine"),
                                "engagement": participant.get("engagement"),
                                "supplement": participant.get("supplement"),
                                "handicap_distance": participant.get("handicapDistance"),
                                "poids_condition_monte_change": participant.get("poidsConditionMonteChange"),
                                "temps_obtenu": participant.get("tempsObtenu"),
                                "temps_obtenu_en_minute": temps_obtenu_formatted,
                                "reduction_kilometrique": participant.get("reductionKilometrique"),
                                "allure": participant.get("allure"),
                            }
                            df_participations = pd.concat([df_participations, pd.DataFrame([participation_data])], ignore_index=True)


                    #------------------------------------------------------------------- Construction de la table participations -------------------------------------------------------------

    else:
        print(f"Échec de la récupération des données, code de statut : {response.status_code}")

    # Passer à la date suivante
    current_date += timedelta(days=1)


In [2]:
df_programmes

,id_programme,date_programme
0,19022013,2013-02-19
1,20022013,2013-02-20
2,21022013,2013-02-21
3,22022013,2013-02-22
4,23022013,2013-02-23
5,24022013,2013-02-24
6,25022013,2013-02-25
7,26022013,2013-02-26
8,27022013,2013-02-27
9,28022013,2013-02-28


In [3]:
print(df_reunions)

    id_reunion id_programme  num_officiel        nature code_hippodrome  \
0            1     19022013             1        DIURNE             VIN   
1            2     19022013             4      NOCTURNE             CAG   
2            3     19022013             5  SEMINOCTURNE             HWM   
3            4     19022013            10        DIURNE             LIG   
4            5     20022013             3  SEMINOCTURNE             ENG   
5            6     20022013             5        DIURNE             XKD   
6            7     20022013             6        DIURNE            None   
7            8     21022013             1        DIURNE             VIN   
8            9     21022013             2        DIURNE             BOR   
9           10     21022013             3  SEMINOCTURNE             GAL   
10          11     21022013             4        DIURNE            None   
11          12     21022013             5  SEMINOCTURNE             DUB   
12          13     220220

In [4]:
df_courses

,id_course,id_reunion,libelle,libelle_court,heure_depart,parcours,distance,distance_unit,corde,discipline,...,deuxieme,montant_offert_2eme,troisieme,montant_offert_3eme,quatrieme,montant_offert_4eme,cinquieme,montant_offert_5eme,incidents_type,incidents_participants
0,1,1,PRIX DU CHESNAY,DU CHESNAY,13:50:00,,2850,METRE,CORDE_GAUCHE,ATTELE,...,5,20000,15,10400,3,4800,11,2400,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,"2, 6, 7, 10"
1,2,1,PRIX DE CHALLANS,DE CHALLANS,14:22:00,,2700,METRE,CORDE_GAUCHE,ATTELE,...,9,10000,6,5200,8,2400,1,1200,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,10
2,3,1,PRIX DE LENS,DE LENS,14:50:00,,2700,METRE,CORDE_GAUCHE,MONTE,...,2,15000,3,7800,7,3600,10,1800,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,"1, 6, 8, 9"
3,4,1,PRIX DE CARHAIX,DE CARHAIX,15:20:00,,2175,METRE,CORDE_GAUCHE,ATTELE,...,6,8250,8,4290,5,1980,2,990,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,"1, 3, 7"
4,5,1,PRIX DE CERISY,DE CERISY,15:55:00,,2700,METRE,CORDE_GAUCHE,MONTE,...,2,11500,11,5980,1,2760,5,1380,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,"4, 9, 12"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,350,54,PRIX DU ROMARIN,DU ROMARIN,20:20:00,,2700,METRE,CORDE_DROITE,MONTE,...,7,4250,10,2210,13,1020,3,510,NON_PARTANT | DISQUALIFIE_POUR_ALLURE_IRREGULIERE,"8 | 2, 6, 9"
350,351,54,PRIX DES PINS,DES PINS,20:50:00,,2050,METRE,CORDE_DROITE,ATTELE,...,8,3250,2,1690,1,780,7,390,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,12
351,352,54,PRIX DES ERABLES,DES ERABLES,21:25:00,,2050,METRE,CORDE_DROITE,ATTELE,...,9,3250,11,1690,5,780,12,390,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,2
352,353,54,PRIX DES CHARMILLES,DES CHARMILLES,21:55:00,,2700,METRE,CORDE_DROITE,ATTELE,...,12,3500,10,1820,11,840,4,420,DISQUALIFIE_POUR_ALLURE_IRREGULIERE,3


In [5]:
df_participations

,id_participation,id_course,nom,numero_cheval,age,sexe,race,statut,proprietaire,entraineur,...,ordre_arrivee,jument_pleine,engagement,supplement,handicap_distance,poids_condition_monte_change,temps_obtenu,temps_obtenu_en_minute,reduction_kilometrique,allure
0,1,1,RODGERS,1,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,Ecurie PELHEM,M. IZAAC,...,9,False,False,0,2850,False,213700,3m 33s,710000,TROT
1,2,1,RAVANELLO,2,8,HONGRES,TROTTEUR FRANCAIS,PARTANT,CH. NEIRINCK,V. MARTENS,...,None,False,False,0,2850,False,None,0m 0s,None,TROT
2,3,1,SERGENT DU RIB,3,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,Ecurie RIB,J.L.CL. DERSOIR,...,4,False,False,0,2850,False,213110,3m 33s,708000,TROT
3,4,1,SEGUINEL MABON,4,7,MALES,TROTTEUR FRANCAIS,PARTANT,Y. DREUX,Y. DREUX,...,1,False,False,0,2850,False,211830,3m 31s,703000,TROT
4,5,1,SILVER DES PRES,5,7,HONGRES,TROTTEUR FRANCAIS,PARTANT,B. HELIAS,F. SOULOY,...,2,False,False,0,2850,False,212470,3m 32s,706000,TROT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4599,4600,354,TITAN DE LA PIERRE,12,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,Mlle M. GOBET,G. RAFFESTIN,...,1,False,False,0,2725,False,211680,3m 31s,737000,TROT
4600,4601,354,TOPAZE DES ONDES,13,6,FEMELLES,TROTTEUR FRANCAIS,PARTANT,A. LANDAIS,L. LAMAZIERE,...,8,False,False,0,2725,False,215230,3m 35s,750000,TROT
4601,4602,354,TOP MARCEAUX,14,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,J. BRUN,G. LAURENT,...,None,False,False,0,2725,False,None,0m 0s,None,TROT
4602,4603,354,TAMACHO MAG,15,6,HONGRES,TROTTEUR FRANCAIS,PARTANT,M. AGOSTINI,P. CALLIER,...,5,False,False,0,2725,False,213280,3m 33s,743000,TROT
